In [69]:
import fingertips_py as ftp
from json import loads
import pandas as pd
from urllib.request import urlopen

In [70]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

- A unique data set is ['Indicator Name', 'Indicator ID', 'Sex', 'Age', 'Time period', 'AreaTypeId]

- Need to create template table of indictor + area codes so that shape shows in QS

- The indicator name needs to be name + id

I need to know for each indicator, which area types and year conifgurations exist so that I know which ones I can pick from and minimise computation for data extraction. - Need to see if I can get a list of this

The alternative would be to create a hierarchy of preference and iterate through the data for each hierachy level. If an indicator is found within a level, then it will no longer be queried for further levels. This might still be computationally expensive but needs to be investigated further 

- time period
- sex
- age

# time period, sex and age available for each indicator

# Area types available for each indicator

In [71]:
raw_available_indicator_at_area = pd.read_json('https://fingertips.phe.org.uk/api/available_data').fillna(value='null')

# Indicator metadata
To get the name of each indicator

In [72]:
indicator_metadata = pd.read_csv('https://fingertips.phe.org.uk/api/indicator_metadata/csv/all')
# indicator_metadata.columns = indicator_metadata.columns.str.lower().str.replace(' ', '_')


# Indicator sex, age and timeperiod available

This doesn't actually bring through all of those available in the data. It brings through the settings for the website and what is shown on the website

In [73]:
def df_nested_json(url): 
    with urlopen(url) as response:
        data_str = response.read().decode("utf-8")
        data_json = loads(data_str[1:-1])
    return pd.json_normalize(data_json)

In [74]:
data = df_nested_json('https://fingertips.phe.org.uk/api/grouproot_summaries/by_indicator_id?indicator_ids=90813')

In [75]:
data

,IndicatorName,IID,GroupId,StateSex,StateAge,StateYearRange,TimePeriod,YearRange,Sex.Id,Sex.Name,Age.Id,Age.Name,Age.Sequence,Category.Id,Category.CategoryTypeId,Category.Name,Category.ShortName,DateChanges.HasDataChangedRecently,DateChanges.DateOfLastChange,DataQualityNote.Id,DataQualityNote.Text,DataQualityNote.Comment,Unit.Id,Unit.Value,Unit.Label,ValueType.Id,ValueType.Name
0,Hospital admissions as a result of self-harm (...,90813,1938132923,False,False,False,2021/22,1,4,Persons,305,10-24 yrs,17.0,-1,-1,Undefined,Undefined,False,07 Mar 2023,0,None,None,3,100000.0,"per 100,000",1,Directly standardised rate


# Area Ids metadata
To get the name of the area

In [76]:
area_ref = pd.read_json('https://fingertips.phe.org.uk/api/area_types').fillna(value='null')

## hierarchy

In [77]:
area_hierarchy = pd.read_csv('https://docs.google.com/spreadsheets/u/0/d/15RhWWsHPPMLWoxR5sJcpK-vraRkidRY8jsAb_Y_5GwI/gviz/tq?tqx=out:csv&tq&gid=963757659&headers=1')
area_hierarchy = area_hierarchy[['AreaTypeId', 'map_position', 'hierarchy']].dropna()

# Area types available for each indicator with extra metadata

## prepare full table

In [78]:
available_indicator_at_area = pd.merge(left=raw_available_indicator_at_area, right=area_ref,
         how='left', left_on='AreaTypeId', right_on='Id',
         indicator=True).rename({'_merge': 'area_merge'}, axis=1)

available_indicator_at_area = pd.merge(left=available_indicator_at_area, right=indicator_metadata.loc[:,['Indicator ID', 'Indicator']],
                                       left_on='IndicatorId', right_on='Indicator ID', how='inner',
                                       indicator=True, )

## use hierarchy to choose indicators and areas

In [79]:
full_area_indicator_hierarchy = pd.merge(left=area_hierarchy, right=raw_available_indicator_at_area,
         left_on='AreaTypeId', right_on='AreaTypeId',
         how='inner')

In [80]:
left = full_area_indicator_hierarchy[full_area_indicator_hierarchy['map_position']=='left']
left['rank'] = left.groupby('IndicatorId')['hierarchy'].rank(method='dense')
keep_left = left[left['rank']==1]


In [81]:
keep_left['rank'].unique()

array([1.])

In [82]:
def find_first_hierarchy(data, map_position):
    data = data[data['map_position']==map_position].loc[:, ['IndicatorId','hierarchy','AreaTypeId', 'map_position']]
    data['rank'] = data.groupby('IndicatorId')['hierarchy'].rank(method='dense')
    return data[data['rank']==1].drop('rank', axis=1)

In [83]:
get_indicator_area = pd.concat([find_first_hierarchy(full_area_indicator_hierarchy, 'left'),
                                find_first_hierarchy(full_area_indicator_hierarchy, 'middle'),
                                find_first_hierarchy(full_area_indicator_hierarchy, 'right')]
)

# Retrieve indicator data

## filter by those that only have all 3 as a test case

In [84]:
number_of_maps_returned = get_indicator_area.groupby('IndicatorId')['map_position'].count()
ind_ids_keep_3_maps = number_of_maps_returned[number_of_maps_returned==3].index.values

In [85]:
get_ind_area_combos = get_indicator_area[get_indicator_area['IndicatorId'].isin(ind_ids_keep_3_maps)]

## get data

In [86]:
def return_ind_list_for_area_type(data, AreaTypeId):
    return data[data['AreaTypeId']==AreaTypeId]['IndicatorId'].values

In [87]:
return_ind_list_for_area_type(get_ind_area_combos, 221)

array([], dtype=int64)

In [88]:
def get_data(data, area_type_id):
    indicator_list = return_ind_list_for_area_type(data, area_type_id)
    
    replace_comma = '%2C'
    ids_as_str = replace_comma.join([str(i) for i in indicator_list])
    print(ids_as_str)
    data = ftp.retrieve_data.get_data_by_indicator_ids(indicator_ids=ids_as_str, # [Maximum 100]
                                                    area_type_id=area_type_id,
                                                    # parent_area_type_id=15,
                                                    # profile_id=None,
                                                    include_sortable_time_periods=True,
                                                    is_test=False)
    
    return data
    
    # data.to_csv(str(area_type_id)+'.csv')

In [89]:
# for area_id in get_ind_area_combos['AreaTypeId'].unique():
#     get_data(get_ind_area_combos, area_id)

# Analysis of indicator ids and names

Some indicator ids have the same name - do they have the same metadata?

No, they have different metadata so a unique indicator would be (indicator id + name)

In [90]:
indicator_ids_per_name = pd.DataFrame(indicator_metadata.groupby('Indicator')['Indicator ID'].nunique()).reset_index()


In [91]:
indicators_with_same_name = indicator_ids_per_name[indicator_ids_per_name['Indicator ID']>1]

In [92]:
indicators_with_same_name.head()

,Indicator,Indicator ID
69,Admission episodes for alcohol-specific condit...,2
96,Admissions for gastroenteritis - CCG,2
107,Adults in contact with secondary mental health...,2
247,"Chlamydia detection rate per 100,000 aged 15 t...",2
402,Estimated prevalence of CHD (55-79 yrs),2


In [93]:
indicator_name = indicators_with_same_name['Indicator'].iloc[0]
indicator_metadata[indicator_metadata['Indicator']==indicator_name]

,Indicator ID,Indicator,Definition,Rationale,Data source,Indicator source,Methodology,Standard population/values,Confidence interval details,Source of numerator,Definition of numerator,Source of denominator,Definition of denominator,Disclosure control,Caveats,Copyright,Data re-use,Links,Indicator number,Notes,Frequency,Rounding,Indicator Content,Specific rationale,Impact of COVID-19,Unit,Value type,Year type,Polarity,Date updated
816,92904,Admission episodes for alcohol-specific condit...,Admissions to hospital for under 18s where the...,Alcohol consumption is a contributing factor t...,Calculated by OHID: Population Health Analysis...,NaN,"Crude rates were calculated per 100,000.",NaN,NaN,Calculated by OHID: Population Health Analysis...,The number of hospital admission episodesfor u...,Office for National Statistics (ONS).,ONS mid-year population estimates for 0-17 yea...,Primary suppression if 5 or less (but not incl...,Hospital admission data can be coded different...,Hospital Episode Statistics (HES) Copyright © ...,NaN,Wider Impact of COVID tool: https://eur03.safe...,13,This indicator is based on admission episodes ...,Annual.,All rates rounded to 2 decimal places.,NaN,NaN,The COVID-19 pandemic had a large impact on ho...,"per 100,000",Crude rate,Financial,RAG - Low is good,27/01/2022
1166,93492,Admission episodes for alcohol-specific condit...,Admissions to hospital for under 18s where the...,Alcohol consumption is a contributing factor t...,Calculated by OHID using data from NHS Digital...,NaN,Admissions to hospital where the primary diagn...,NaN,NaN,Calculated by OHID using datausing data fromNH...,The number of hospital admission episodesfor u...,Hospital Episode Statistics (HES),CCG populations are aggregated from NHS Digita...,All sub-national counts are rounded to the nea...,Hospital admission data can be coded different...,Copyright © NHS Digital. All rights reserved.\...,NaN,https://analytics.phe.gov.uk/apps/covid-19-ind...,NaN,This indicator is based on admission episodes ...,Annual.,All rates rounded to 2 decimal places.,NaN,NaN,The COVID-19 pandemic had a large impact on ho...,"per 100,000",Crude rate,Financial,RAG - Low is good,16/02/2022
